# Task 4:

1.	Download Alice in Wonderland by Lewis Carroll from Project Gutenberg's website http://www.gutenberg.org/files/11/11-0.txt


2.	Perform any necessary preprocessing on the text, including converting to lower case, removing stop words, numbers / non-alphabetic characters, lemmatization.


3.	Find Top 10 most important (for example, in terms of TF-IDF metric) words from each chapter in the text (not "Alice"); how would you name each chapter according to the identified tokens?


4.	Find the Top 10 most used verbs in sentences with Alice. What does Alice do most often?


5.	*(not necessary) Find Top 100 most used verbs in sentences with Alice. Get word vectors using a pre-trained word2vec model and visualize them. Compare the words using embeddings. 


In [15]:
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer, WhitespaceTokenizer

In [16]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\belpa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\belpa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\belpa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\belpa\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# 1-2.	Download Alice in Wonderland by Lewis Carroll and Perform any necessary preprocessing on the text, including converting to lower case, removing stop words, numbers / non-alphabetic characters, lemmatization.

In [83]:
file = open('AliceInWonderland.txt', 'r', encoding = 'utf-8')
text = file.read()
file.close()

text[:2000]

'The Project Gutenberg eBook of Alice’s Adventures in Wonderland, by Lewis Carroll\n\nThis eBook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this eBook or online at\nwww.gutenberg.org. If you are not located in the United States, you\nwill have to check the laws of the country where you are located before\nusing this eBook.\n\nTitle: Alice’s Adventures in Wonderland\n\nAuthor: Lewis Carroll\n\nRelease Date: January, 1991 [eBook #11]\n[Most recently updated: October 12, 2020]\n\nLanguage: English\n\nCharacter set encoding: UTF-8\n\nProduced by: Arthur DiBianca and David Widger\n\n*** START OF THE PROJECT GUTENBERG EBOOK ALICE’S ADVENTURES IN WONDERLAND ***\n\n[Illustration]\n\n\n\n\nAlice’s Adventures in Wonderland\n\nby Lewis Carroll\n\nTHE MILLENNIUM FULCRUM EDITION 3.0\n\nConten

In [84]:
# Deleting the header and the footer in text

text = text.split('CHAPTER I.')[2]
text = text.split('THE END')[0]

text[:2000] # Now we got only text of book chapters

'\nDown the Rabbit-Hole\n\n\nAlice was beginning to get very tired of sitting by her sister on the\nbank, and of having nothing to do: once or twice she had peeped into\nthe book her sister was reading, but it had no pictures or\nconversations in it, “and what is the use of a book,” thought Alice\n“without pictures or conversations?”\n\nSo she was considering in her own mind (as well as she could, for the\nhot day made her feel very sleepy and stupid), whether the pleasure of\nmaking a daisy-chain would be worth the trouble of getting up and\npicking the daisies, when suddenly a White Rabbit with pink eyes ran\nclose by her.\n\nThere was nothing so _very_ remarkable in that; nor did Alice think it\nso _very_ much out of the way to hear the Rabbit say to itself, “Oh\ndear! Oh dear! I shall be late!” (when she thought it over afterwards,\nit occurred to her that she ought to have wondered at this, but at the\ntime it all seemed quite natural); but when the Rabbit actually _took a\nwatch 

In [85]:
# Removing extra characters with replacement

text = re.sub('<.*?>', ' ', text)
text = re.sub(' +', ' ', text)
text = re.sub(r'[^\w\s]', '', text)
text = re.sub('\n', '', text)

text[:2000]

'Down the RabbitHoleAlice was beginning to get very tired of sitting by her sister on thebank and of having nothing to do once or twice she had peeped intothe book her sister was reading but it had no pictures orconversations in it and what is the use of a book thought Alicewithout pictures or conversationsSo she was considering in her own mind as well as she could for thehot day made her feel very sleepy and stupid whether the pleasure ofmaking a daisychain would be worth the trouble of getting up andpicking the daisies when suddenly a White Rabbit with pink eyes ranclose by herThere was nothing so _very_ remarkable in that nor did Alice think itso _very_ much out of the way to hear the Rabbit say to itself Ohdear Oh dear I shall be late when she thought it over afterwardsit occurred to her that she ought to have wondered at this but at thetime it all seemed quite natural but when the Rabbit actually _took awatch out of its waistcoatpocket_ and looked at it and then hurriedon Alice st

In [86]:
len(text)

132114

# 3.	Find Top 10 most important (for example, in terms of TF-IDF metric) words from each chapter in the text (not "Alice"); how would you name each chapter according to the identified tokens?

In [87]:
# Splitting the text by the chapters

chapters = text.split('CHAPTER')
len(chapters)

12

In [88]:
# Lemmatization + removing stop-words:

text = re.sub(r'[^\w\s]', '', text, re.UNICODE)
text = text.lower()

stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

text = [lemmatizer.lemmatize(token) for token in TreebankWordTokenizer().tokenize(text)]
text = [word for word in text if not word in stop_words]

len(text)

12785

In [89]:
for idx, chapter in enumerate(chapters):
    
    chapter = re.sub(r'[^\w\s]', '', chapter, re.UNICODE)
    chapter = chapter.lower()
    chapter = [lemmatizer.lemmatize(token) for token in TreebankWordTokenizer().tokenize(chapter)]
    chapter = [word for word in chapter if not word in stop_words]
    
    analys = nltk.Text(chapter)
    print('\nTop 10 most important words from chapter {}:'.format(idx + 1))
    
    scores = {word: nltk.TextCollection(text).tf_idf(word, chapter) for word in analys}
    sorted_words = sorted(scores.items(), key = lambda x: x[1], reverse = True)
    counter = 0
    
    for word, score in sorted_words[:11]:
        if word == 'wa':
            word = 'was'
        if word != 'alice':
            print("\tWord: {}, TF-IDF metric: {}".format(word, round(score, 5)))
            counter += 1
        if counter == 9:
            continue


Top 10 most important words from chapter 1:
	Word: was, TF-IDF metric: 0.14649
	Word: little, TF-IDF metric: 0.06388
	Word: like, TF-IDF metric: 0.05322
	Word: see, TF-IDF metric: 0.04594
	Word: bat, TF-IDF metric: 0.04386
	Word: way, TF-IDF metric: 0.04345
	Word: think, TF-IDF metric: 0.042
	Word: door, TF-IDF metric: 0.04134
	Word: nothing, TF-IDF metric: 0.04114
	Word: key, TF-IDF metric: 0.04098

Top 10 most important words from chapter 2:
	Word: was, TF-IDF metric: 0.08917
	Word: mouse, TF-IDF metric: 0.06743
	Word: little, TF-IDF metric: 0.06666
	Word: dear, TF-IDF metric: 0.05204
	Word: cried, TF-IDF metric: 0.04989
	Word: foot, TF-IDF metric: 0.04908
	Word: pool, TF-IDF metric: 0.04769
	Word: must, TF-IDF metric: 0.04329
	Word: thing, TF-IDF metric: 0.04182
	Word: oh, TF-IDF metric: 0.04166

Top 10 most important words from chapter 3:
	Word: said, TF-IDF metric: 0.12725
	Word: mouse, TF-IDF metric: 0.11355
	Word: dodo, TF-IDF metric: 0.08253
	Word: was, TF-IDF metric: 0.06269


# 4. Find the Top 10 most used verbs in sentences with Alice. What does Alice do most often?

In [90]:
file = open('AliceInWonderland.txt', 'r', encoding = 'utf-8')
text = file.read()
file.close()

text = text.split('CHAPTER I.')[2]
text = text.split('THE END')[0]
text = re.sub('\n', '', text)

In [91]:
# https://stackoverflow.com/a/31505798 split into sentences code was used:

alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms + " " + starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" " + suffixes + "[.] " + starters," \\1<stop> \\2",text)
    text = re.sub(" " + suffixes + "[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

splitted_text = split_into_sentences(text)
splitted_text[:10]

['Down the Rabbit-HoleAlice was beginning to get very tired of sitting by her sister on thebank, and of having nothing to do: once or twice she had peeped intothe book her sister was reading, but it had no pictures orconversations in it, “and what is the use of a book,” thought Alice“without pictures or conversations?',
 '”So she was considering in her own mind (as well as she could, for thehot day made her feel very sleepy and stupid), whether the pleasure ofmaking a daisy-chain would be worth the trouble of getting up andpicking the daisies, when suddenly a White Rabbit with pink eyes ranclose by her.',
 'There was nothing so _very_ remarkable in that; nor did Alice think itso _very_ much out of the way to hear the Rabbit say to itself, “Ohdear!',
 'Oh dear!',
 'I shall be late!',
 '” (when she thought it over afterwards,it occurred to her that she ought to have wondered at this, but at thetime it all seemed quite natural); but when the Rabbit actually _took awatch out of its waistco

In [92]:
# Searching all sentences with 'Alice':
sentences_with_alice = []

for sentence in splitted_text:
    if re.search(r'\bAlice\b', sentence):
        sentences_with_alice.append(sentence)
        
len(sentences_with_alice)

369

In [93]:
# Searching all verbs in sentences with 'Alice':

def is_verb(tag):
    return tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

a = lambda x: lemmatizer.lemmatize(x[0], 'v')
verbdict = []

for sentence in sentences_with_alice:
    
    tokens = nltk.word_tokenize(sentence)
    pos_tagged = nltk.pos_tag(tokens)
    verbs = filter(lambda x:is_verb(x[1]),pos_tagged)
    verbdict.append((list(map(a, list(verbs)))))
    
verbdict[0]

['be',
 'begin',
 'get',
 'sit',
 'have',
 'do',
 'twice',
 'have',
 'peep',
 'be',
 'read',
 'have',
 'be',
 'think']

In [94]:
verbs = []
for word_arr in verbdict:
    for word in word_arr:
        word = re.sub(r"[^\w\s]", "", word, re.UNICODE)
        if word and word != "s":
            verbs.append(word)
            
verbs[:25]

['be',
 'begin',
 'get',
 'sit',
 'have',
 'do',
 'twice',
 'have',
 'peep',
 'be',
 'read',
 'have',
 'be',
 'think',
 'be',
 'do',
 'think',
 'hear',
 'say',
 'think',
 'occur',
 'have',
 'wonder',
 'seem',
 '_took']

In [95]:
# Top 10 most used verbs in sentences with 'Alice' are:

fdist = FreqDist(nltk.Text(verbs))
fdist.most_common(10)

[('be', 226),
 ('say', 182),
 ('have', 90),
 ('think', 67),
 ('go', 51),
 ('do', 41),
 ('get', 39),
 ('look', 33),
 ('see', 28),
 ('begin', 27)]